Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
intermediate_nodes = 1024
num_steps = 10001
l2_beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables_L1.
  weights_L1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, intermediate_nodes]))
  biases_L1 = tf.Variable(tf.zeros([intermediate_nodes]))
  
  # Variables_L2.
  weights_L2 = tf.Variable(
    tf.truncated_normal([intermediate_nodes, num_labels]))
  biases_L2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation_L1.
  logits_L1 = tf.matmul(tf_train_dataset, weights_L1) + biases_L1

  #Adding ReLU layer
  logits_ReLU = tf.nn.relu(logits_L1)

  # Training computation_L2.
  logits_L2 = tf.matmul(logits_ReLU, weights_L2) + biases_L2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_L2))\
    + l2_beta * (tf.nn.l2_loss(weights_L1) + tf.nn.l2_loss(weights_L2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_L2)
  valid_prediction_L1 = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weights_L1) + biases_L1)
  valid_prediction_L2 = tf.nn.softmax(
    tf.matmul(valid_prediction_L1, weights_L2) + biases_L2)
  test_prediction_L1 = tf.nn.relu(
    tf.matmul(tf_test_dataset, weights_L1) + biases_L1)
  test_prediction_L2 = tf.nn.softmax(
    tf.matmul(test_prediction_L1, weights_L2) + biases_L2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("\t\t\tValidation accuracy: %.1f%%" % accuracy(
        valid_prediction_L2.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_L2.eval(), test_labels))

Initialized
Minibatch loss at step 0: 727.226074
Minibatch accuracy: 8.6%
			Validation accuracy: 37.2%
Minibatch loss at step 500: 194.586838
Minibatch accuracy: 82.0%
			Validation accuracy: 80.1%
Minibatch loss at step 1000: 114.718864
Minibatch accuracy: 82.0%
			Validation accuracy: 81.1%
Minibatch loss at step 1500: 68.972633
Minibatch accuracy: 87.5%
			Validation accuracy: 83.0%
Minibatch loss at step 2000: 41.256111
Minibatch accuracy: 89.8%
			Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.062134
Minibatch accuracy: 92.2%
			Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.446385
Minibatch accuracy: 85.2%
			Validation accuracy: 86.9%
Minibatch loss at step 3500: 9.468951
Minibatch accuracy: 92.2%
			Validation accuracy: 87.1%
Minibatch loss at step 4000: 6.145636
Minibatch accuracy: 80.5%
			Validation accuracy: 87.5%
Minibatch loss at step 4500: 3.682882
Minibatch accuracy: 93.0%
			Validation accuracy: 88.0%
Minibatch loss at step 5000: 2.548502
Minib

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [6]:
batch_size = 128
intermediate_nodes = 1024
num_steps = 10001
l2_beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables_L1.
  weights_L1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, intermediate_nodes]))
  biases_L1 = tf.Variable(tf.zeros([intermediate_nodes]))
  
  # Variables_L2.
  weights_L2 = tf.Variable(
    tf.truncated_normal([intermediate_nodes, num_labels]))
  biases_L2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation_L1.
  logits_L1 = tf.matmul(tf_train_dataset, weights_L1) + biases_L1

  #Adding ReLU layer
  logits_ReLU = tf.nn.relu(logits_L1)

  # Training computation_L2.
  logits_L2 = tf.matmul(logits_ReLU, weights_L2) + biases_L2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_L2))\
    + l2_beta * (tf.nn.l2_loss(weights_L1) + tf.nn.l2_loss(weights_L2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_L2)
  valid_prediction_L1 = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weights_L1) + biases_L1)
  valid_prediction_L2 = tf.nn.softmax(
    tf.matmul(valid_prediction_L1, weights_L2) + biases_L2)
  test_prediction_L1 = tf.nn.relu(
    tf.matmul(tf_test_dataset, weights_L1) + biases_L1)
  test_prediction_L2 = tf.nn.softmax(
    tf.matmul(test_prediction_L1, weights_L2) + biases_L2)

reduced_batches = 2
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step%reduced_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("\t\t\tValidation accuracy: %.1f%%" % accuracy(
        valid_prediction_L2.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_L2.eval(), test_labels))

Initialized
Minibatch loss at step 0: 772.997192
Minibatch accuracy: 7.8%
			Validation accuracy: 24.2%
Minibatch loss at step 500: 191.393509
Minibatch accuracy: 100.0%
			Validation accuracy: 71.9%
Minibatch loss at step 1000: 116.071495
Minibatch accuracy: 100.0%
			Validation accuracy: 71.9%
Minibatch loss at step 1500: 70.392128
Minibatch accuracy: 100.0%
			Validation accuracy: 72.0%
Minibatch loss at step 2000: 42.689651
Minibatch accuracy: 100.0%
			Validation accuracy: 72.1%
Minibatch loss at step 2500: 25.889397
Minibatch accuracy: 100.0%
			Validation accuracy: 72.3%
Minibatch loss at step 3000: 15.700996
Minibatch accuracy: 100.0%
			Validation accuracy: 72.8%
Minibatch loss at step 3500: 9.522803
Minibatch accuracy: 100.0%
			Validation accuracy: 73.3%
Minibatch loss at step 4000: 5.777721
Minibatch accuracy: 100.0%
			Validation accuracy: 74.3%
Minibatch loss at step 4500: 3.509511
Minibatch accuracy: 100.0%
			Validation accuracy: 75.2%
Minibatch loss at step 5000: 2.137

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [7]:
batch_size = 128
intermediate_nodes = 1024
num_steps = 10001
l2_beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables_L1.
  weights_L1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, intermediate_nodes]))
  biases_L1 = tf.Variable(tf.zeros([intermediate_nodes]))
  
  # Variables_L2.
  weights_L2 = tf.Variable(
    tf.truncated_normal([intermediate_nodes, num_labels]))
  biases_L2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation_L1.
  logits_L1 = tf.matmul(tf_train_dataset, weights_L1) + biases_L1
    
  #Adding ReLU layer - with dropout
  dropout = tf.nn.dropout(logits_L1, 0.75)
  logits_ReLU = tf.nn.relu(dropout)

  # Training computation_L2.
  logits_L2 = tf.matmul(logits_ReLU, weights_L2) + biases_L2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_L2))\
    + l2_beta * (tf.nn.l2_loss(weights_L1) + tf.nn.l2_loss(weights_L2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_L2)
  valid_prediction_L1 = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weights_L1) + biases_L1)
  valid_prediction_L2 = tf.nn.softmax(
    tf.matmul(valid_prediction_L1, weights_L2) + biases_L2)
  test_prediction_L1 = tf.nn.relu(
    tf.matmul(tf_test_dataset, weights_L1) + biases_L1)
  test_prediction_L2 = tf.nn.softmax(
    tf.matmul(test_prediction_L1, weights_L2) + biases_L2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("\t\t\tValidation accuracy: %.1f%%" % accuracy(
        valid_prediction_L2.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction_L2.eval(), test_labels))

Initialized
Minibatch loss at step 0: 736.809631
Minibatch accuracy: 10.9%
			Validation accuracy: 29.3%
Minibatch loss at step 500: 193.289215
Minibatch accuracy: 84.4%
			Validation accuracy: 79.6%
Minibatch loss at step 1000: 115.142723
Minibatch accuracy: 79.7%
			Validation accuracy: 81.5%
Minibatch loss at step 1500: 69.213211
Minibatch accuracy: 82.8%
			Validation accuracy: 82.2%
Minibatch loss at step 2000: 41.274532
Minibatch accuracy: 87.5%
			Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.152308
Minibatch accuracy: 86.7%
			Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.466117
Minibatch accuracy: 88.3%
			Validation accuracy: 86.1%
Minibatch loss at step 3500: 9.473433
Minibatch accuracy: 89.8%
			Validation accuracy: 86.8%
Minibatch loss at step 4000: 6.171430
Minibatch accuracy: 79.7%
			Validation accuracy: 87.4%
Minibatch loss at step 4500: 3.705223
Minibatch accuracy: 91.4%
			Validation accuracy: 87.5%
Minibatch loss at step 5000: 2.572432
Mini

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [8]:
batch_size = 64
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 256
num_hidden_nodes3 = 128
keep_prob = 0.5
num_steps = 10001
l2_beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
  weights4 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes3, num_labels], stddev=0.1))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) + biases2)
  lay3_train = tf.nn.relu(tf.matmul(lay2_train, weights3) + biases3)
  logits = tf.matmul(lay3_train, weights4) + biases4
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))\
    + l2_beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.2, global_step,
    2000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
  lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
  lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(lay3_test, weights4) + biases4)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("\t\t\tValidation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.565770
Minibatch accuracy: 12.5%
			Validation accuracy: 25.2%
Minibatch loss at step 500: 2.210745
Minibatch accuracy: 82.8%
			Validation accuracy: 84.9%
Minibatch loss at step 1000: 1.883207
Minibatch accuracy: 84.4%
			Validation accuracy: 84.8%
Minibatch loss at step 1500: 1.609742
Minibatch accuracy: 84.4%
			Validation accuracy: 86.3%
Minibatch loss at step 2000: 1.299401
Minibatch accuracy: 89.1%
			Validation accuracy: 87.3%
Minibatch loss at step 2500: 1.198640
Minibatch accuracy: 89.1%
			Validation accuracy: 88.1%
Minibatch loss at step 3000: 0.971841
Minibatch accuracy: 93.8%
			Validation accuracy: 88.0%
Minibatch loss at step 3500: 0.870794
Minibatch accuracy: 96.9%
			Validation accuracy: 88.2%
Minibatch loss at step 4000: 0.843518
Minibatch accuracy: 92.2%
			Validation accuracy: 88.3%
Minibatch loss at step 4500: 0.822466
Minibatch accuracy: 92.2%
			Validation accuracy: 88.6%
Minibatch loss at step 5000: 0.756837
Minibatch accu